In [26]:
import os
import dask.dataframe as dd
import polars as pl
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

polars.config.Config

In [27]:
#load events_df.parquet
events_df = dd.read_parquet('MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet')
events_df = events_df.compute()

In [28]:
events_df

,event_id,subject_id,timestamp,event_type,age,time_of_day
0,0,0,2125-10-18 16:50:00,ADMISSION,71.793844,PM
1,1,0,2125-10-20 15:15:00,DISCHARGE&DIAGNOSIS,71.799139,PM
2,2,0,2125-10-26 20:32:00,ADMISSION,71.816169,PM
3,3,0,2125-10-27 16:28:00,DISCHARGE&DIAGNOSIS,71.818442,PM
4,4,0,2125-11-30 16:02:00,ADMISSION,71.911480,PM
...,...,...,...,...,...,...
14694383,14694383,12126,2164-09-17 15:17:32,LAB,58.712217,PM
14694384,14694384,12126,2164-09-17 15:22:00,INFUSION_END,58.712225,PM
14694385,14694385,12126,2164-09-17 16:34:00,INFUSION_END,58.712362,PM
14694386,14694386,12126,2164-09-17 16:35:00,PROCEDURE_END,58.712364,PM


In [29]:
#load dynamic_measurements_df.parquet
dynamic_measurements_df = dd.read_parquet('MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet')

In [30]:
sub_dynamic_measurements_df = dynamic_measurements_df[['event_id', 'diagnosis', 'lab', 'valuenum']]
sub_dynamic_measurements_df = sub_dynamic_measurements_df.compute()

In [33]:
sub_dynamic_measurements_df = sub_dynamic_measurements_df.set_index('event_id')

In [35]:
sub_dynamic_measurements_df.dropna(inplace=True, how='all')

In [36]:
sub_dynamic_measurements_df

,diagnosis,lab,valuenum
event_id,,,
1889500,ICD9CM/1629,NaN,NaN
1888856,UNK,NaN,NaN
1889508,ICD9CM/431,NaN,NaN
1889508,ICD9CM/1983,NaN,NaN
1889504,ICD9CM/1629,NaN,NaN
...,...,...,...
14694381,NaN,Ionized Calcium (mmol/L),1.15
14694383,NaN,Sodium (whole blood) (mEq/L),134.00
14694383,NaN,Potassium (whole blood) (mEq/L),4.20


In [37]:
df_result = events_df.merge(sub_dynamic_measurements_df, how='left', on='event_id')

In [39]:
df_result = df_result.drop(['event_id', 'time_of_day'], axis=1)

In [40]:
df_result.sort_values(by=['subject_id', 'timestamp'], inplace=True)

In [43]:
df_result.drop_duplicates(subset=['subject_id', 'timestamp', 'event_type'], keep='first', inplace=True)

In [45]:
df_result.reset_index(drop=True, inplace=True)

In [47]:
df_result = pl.DataFrame(df_result)

In [48]:
df_result

subject_id,timestamp,event_type,age,diagnosis,lab,valuenum
u16,datetime[μs],cat,f64,cat,cat,f64
0,2125-10-18 16:50:00,"""ADMISSION""",71.793844,null,null,null
0,2125-10-20 15:15:00,"""DISCHARGE&DIAG…",71.799139,"""UNK""",null,null
0,2125-10-26 20:32:00,"""ADMISSION""",71.816169,null,null,null
0,2125-10-27 16:28:00,"""DISCHARGE&DIAG…",71.818442,"""ICD9CM/56211""",null,null
0,2125-11-30 16:02:00,"""ADMISSION""",71.91148,null,null,null
0,2125-12-03 14:44:00,"""DISCHARGE&DIAG…",71.919545,"""ICD9CM/49122""",null,null
0,2125-12-26 17:06:00,"""ADMISSION""",71.982786,null,null,null
0,2125-12-27 16:52:00,"""DISCHARGE&DIAG…",71.985497,"""ICD9CM/4019""",null,null
0,2126-11-04 00:50:00,"""ADMISSION""",72.837877,null,null,null


In [49]:
df_result.write_parquet('MIMIC_ESD_new_schema_08-31-23-1/data.parquet')